# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.



## Part 1

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.

1. Expand rows and cols based on empty rows and cols
2. Detect galaxies
3. Combinations of galaxiies pairs
4. L1 distance to galaxies




In [7]:
from __future__ import annotations
from dataclasses import dataclass
from typing import NamedTuple
from itertools import combinations

class Galaxy(NamedTuple):
    x:int
    y:int


    @staticmethod
    def parse_galaxy(char:str, x:int, y:int)->Galaxy:
        if char == '#':
            return Galaxy(x=x, y=y)
        
    def l1_distance(self, galaxy:Galaxy)->int:
        """
        Calculate the L1 distance (Manhattan distance) between two points in 2D space.
        
          
        Returns:
        l1_distance : int - L1 distance between the two points
        """
        x1, y1 = self.x, self.y
        x2, y2 = galaxy.x, galaxy.y
        
        return abs(x1 - x2) + abs(y1 - y2)

        
    

@dataclass
class Universe:
    raw:str

    def __post_init__(self):
        self._empty_rows()       
        self._empty_cols()
        self.expand_universe()

    def _empty_rows(self):
        rows = self.raw.splitlines()
        xmax, ymax = len(rows[0]), len(rows)
        empty_rows = list()
        for y in range(0, ymax):
            row_empty = all(rows[y][x] == '.' 
                    for x in range(0, xmax))
            if row_empty:
                empty_rows.append(y)
        self.empty_rows = empty_rows

    def _empty_cols(self):
        rows = self.raw.splitlines()
        xmax, ymax = len(rows[0]), len(rows)
        empty_cols = list()
        for x in range(0, xmax):
            col_empty = all(rows[y][x] == '.'
                for y in range(0, ymax)
            )
            if col_empty:
                empty_cols.append(x)
        self.empty_cols = empty_cols

    def expand_universe(self):
        # Iterate through each row and insert '.' at specified positions
        rows = self.raw.splitlines()
        for i in range(len(rows)):
            row = list(rows[i])
            # print('start', row)
            # need to account for location shift after insertion
            for ii, pos in enumerate(self.empty_cols):
                row.insert(pos+ii, '.' )
                 # print(pos, row)
            rows[i] = ''.join(row)
        xmax = len(rows[0])
        for ii, pos in enumerate(self.empty_rows):
            rows.insert(pos+ii, '.' * xmax)
        self.expanded_universe = '\n'.join(rows)
        
    def parse_universe(self, part1=True)->Universe:
        galaxies  = list()
        rows = self.expanded_universe.splitlines() if part1 else self.raw.splitlines()
        for y, row in enumerate(rows):
            for x, char in enumerate(row):
                char = rows[y][x]
                if char != '.':
                    galaxies.append(Galaxy.parse_galaxy(char, x=x, y=y))
        self.galaxies = galaxies


    def shortest_paths(self)->int:
        # comb ignores order repeats
        pairs = combinations(self.galaxies, 2) 
        return sum(
            g1.l1_distance(g2)
            for g1, g2 in pairs
        )
    

    def shortest_paths_p2(self, factor:int=2)->int:
        # comb ignores order repeats
        pairs = combinations(self.galaxies, 2)

        total = 0
        for g1, g2 in pairs:
            ymin, ymax = min(g1.y, g2.y), max(g1.y, g2.y)
            xmin, xmax = min(g1.x, g2.x), max(g1.x, g2.x)
            for y in range(ymin, ymax): 
                if y in self.empty_rows:
                    total += factor # increase by expansion factor
                else:
                    total += 1 # increase by regular l1
            for x in range(xmin, xmax):
                if x in self.empty_cols:
                    total += factor 
                else:
                    total += 1 
        return total


def part1(puzzle:str)->int:
    universe  = Universe(raw=puzzle)
    universe.parse_universe()
    return universe.shortest_paths()


def part2(puzzle:str, factor:int)->int:
    universe  = Universe(raw=puzzle)
    universe.parse_universe(part1=False)
    # print(len(universe.galaxies))
    return universe.shortest_paths_p2(factor=factor)

TEST = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

TEST2 = """....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#......."""

In [10]:
assert part1(puzzle=TEST) == 374
assert part2(puzzle=TEST, factor=2) == 374
assert part2(puzzle=TEST, factor=10) == 1030
assert part2(puzzle=TEST, factor=100) == 8410

## Solutions

In [9]:
with open("puzzle_input/day11.txt") as file:
    puzzle = file.read()
print(part1(puzzle=puzzle))
print(part2(puzzle=puzzle, factor=1_000_000))


9565386
857986849428
